In [1]:
import pandas as pd
meta = pd.read_csv("meta_lyrik.csv", encoding = "utf8")
meta.head()

,Unnamed: 0,id,author,title,genre,date,file,period,tokens_cleaned
0,0,1,"Ebeling, Johann Justus",N.A.,lyrik,1747,dta.poem.1.txt,Aufklaerung,333
1,1,2,"Ebeling, Johann Justus",Der Sommer.,lyrik,1747,dta.poem.2.txt,Aufklaerung,3136
2,2,3,"Ebeling, Johann Justus",Die mannigfaltige \n Weisheit GOttes \n im ...,lyrik,1747,dta.poem.3.txt,Aufklaerung,196
3,3,4,"Ebeling, Johann Justus",Die \n angenehme Morgenröthe \n Und \n da...,lyrik,1747,dta.poem.4.txt,Aufklaerung,768
4,4,5,"Ebeling, Johann Justus",Anrede an den herrlichen GOtt \n um Abwendung...,lyrik,1747,dta.poem.5.txt,Aufklaerung,203


In [2]:
def load_corpus(path, df):
    from numpy import append 

    texts = []
    filenames = []
    for filename in df["file"]:
        with open(path + filename, 'r', encoding="utf16") as f:
            texts.append(f.read())
            f.close()
        filenames.append(filename)    
    return texts, filenames

In [5]:
def load_corpus_lyrik(path):
    from numpy import append 
    from os import listdir
    
    texts = []
    filenames = []
    for filename in listdir(path):
        with open(path + "/" + filename, 'r', encoding="utf8") as f:
            texts.append(f.read())

        filenames.append(filename)    
    return texts, filenames

In [6]:
texts, filenames = load_corpus_lyrik("corpora/cleaned_normalized/corpus_lyrik/")

In [ ]:
sent = [text.split() for text in texts]

In [ ]:
from gensim.models.phrases import Phrases, Phraser
import gensim.models
import multiprocessing

In [ ]:
def train_model(words):
    phrases = Phrases(words, min_count=30, progress_per=10000) # detects multi word expressions

    bigram = Phraser(phrases)
    sentences = bigram[words]
    
    model = gensim.models.FastText(min_count=MIN_COUNT,
                                   vector_size=VECTOR_SIZE,
                                   window = WINDOW,
                                   alpha=ALPHA,
                                   workers = multiprocessing.cpu_count()-1,
                                   negative = NEGATIVE,
                                   seed = 42)

    model.build_vocab(corpus_iterable=sentences)

    model.train(corpus_iterable=sentences,
                        epochs=EPOCHS,
                        total_examples=model.corpus_count)
    
    return model

In [ ]:
def model_evaluation(lang_model):
    odd_one_out =["adel volk republik monarchie gericht wohlstand", #staatstheorie wohlstand
                  "gedicht roman buch urheber druck roman gesetz", #schriftstellertum gesetz
                  "haus wohlstand reichtum handwerk familie könig", #bürgertum könig
                  "philosophie verstand gelassenheit ethik ruhe zorn", #vernunft zorn
                  "trauer wut freude glück zorn verstand", #gefühle verstand
                  "mutter königin geburt maid künstlerin krieger", #frauenbild krieger
                  "bild lied statur gedicht künstler acker", #kunst acker 
                  "ritter schild jäger maid jungfer pistole", #vergangeheit pistole
                  "dämon fee drache magie zauberer krankheit", #phantastisch krankheit
                  "gott heilig priester kloster pfaffe schmied", #religion schmied
                  "fluss gebirge sturm blume ozean mine", #natur mine
                  "angst monster geister friedhof wahnsinn sonne", #nacht sonne
                  "ausbildung reife erwachsen erkenntnis lebensweg kind" #bildung kind
                  ] 

    odd_one_out = [string.split() for string in odd_one_out]

    odd_one = ["wohlstand",
               "gesetz",
               "könig",
               "zorn",
               "verstand",
               "krieger",
               "acker",
               "pistole",
               "krankheit",
               "schmied",
               "mine",
               "sonne",
               "kind"]

    correct = 0
    for i in range(0,len(odd_one_out)):
        if lang_model.wv.doesnt_match(odd_one_out[i]) == odd_one[i]:
            correct += 1

    return correct/len(odd_one_out)

In [ ]:
# 144 parameter

min_count = [10, 20]
vector_size = [200, 300]
window = [5, 15, 20]
epochs = [30, 50]
negative = [5, 10, 20]
alpha = [0.01, 0.1]

In [ ]:
# f = open("finetuning.csv", "w")
# f.write("min_count,vector_size,window,epoch,negative,alpha,score\n")
# f.close()

In [ ]:
i = 93
for MIN_COUNT in min_count:
    for VECTOR_SIZE in vector_size:
        for WINDOW in window:
            for EPOCHS in epochs:
                for NEGATIVE in negative:
                    for ALPHA in alpha:
                        i += 1
                        if i < 1:
                            continue
                        else:    
                            print(i)
                            model = train_model(sent)
                            score = model_evaluation(model)
                            f = open("finetuning.csv", "a")
                            f.write(str(MIN_COUNT) + "," + str(VECTOR_SIZE) + "," + str(WINDOW) + "," + str(EPOCHS) + "," + str(NEGATIVE) + "," + str(ALPHA) + "," + str(score) + "\n")
                            f.close()                  